In [36]:
using Random

function randomized_hill_climbing(initial_state, max_iterations, neighbor_func, energy_func)
    current_state = initial_state
    current_energy = energy_func(current_state)
    
    for i in 1:max_iterations
        # generate a random neighbor state
        neighbor_state = neighbor_func(current_state)
        neighbor_energy = energy_func(neighbor_state)
        
        # move to the neighbor state if its energy is better
        if neighbor_energy < current_energy
            current_state = neighbor_state
            current_energy = neighbor_energy
        end
    end
    
    return current_state, current_energy
end


using Random

function simulated_annealing(initial_state, temperature, cooling_rate, num_iterations, energy_func, neighbor_func)
    current_state = initial_state
    best_state = initial_state
    current_energy = energy_func(current_state)
    best_energy = current_energy
    
    for i in 1:num_iterations
        # generate a neighbor state
        neighbor_state = neighbor_func(current_state)
        neighbor_energy = energy_func(neighbor_state)
        
        # calculate probability of accepting neighbor state
        delta_energy = neighbor_energy - current_energy
        if delta_energy < 0 || rand(Float64) < exp(-delta_energy / temperature)
            current_state = neighbor_state
            current_energy = neighbor_energy
            
            # update best state if necessary
            if current_energy < best_energy
                best_state = current_state
                best_energy = current_energy
            end
        end
        
        # reduce temperature
        temperature *= cooling_rate
    end
    
    return best_state, best_energy
end


using Random

function genetic_algorithm(pop_size, n_genes, fitness_func, mutation_rate, num_generations)
    
    population = rand(Bool, pop_size, n_genes)
    for i in 1:num_generations
        # evaluate fitness of each individual in the population
        fitness_values = [fitness_func(population[i,:]) for i in 1:pop_size]
        
        # select parents for next generation
        #parent_indices = sample(1:pop_size, 2, replace=false, weights=fitness_values)
        #parent1 = population[parent_indices[1], :]
        #parent2 = population[parent_indices[2], :]
        parent_indices = randperm(pop_size)[1:2]
        parent1 = population[parent_indices[1], :]
        parent2 = population[parent_indices[2], :]
        
        # perform crossover to generate new individual
        crossover_point = rand(1:n_genes-1)
        child = vcat(parent1[1:crossover_point], parent2[crossover_point+1:end])
        
        # perform mutation on child
        for j in 1:n_genes
            if rand() < mutation_rate
                child[j] = !child[j]
            end
        end
        
        # replace least fit individual in population with child
        min_fitness_index = argmin(fitness_values)
        population[min_fitness_index, :] = child
    end
    
    # evaluate fitness of final population
    fitness_values = [fitness_func(population[i,:]) for i in 1:pop_size]
    
    # return best individual found
    best_index = argmax(fitness_values)
    best_individual = population[best_index, :]
    best_x = [i == true ? 1 : 0 for i in best_individual]
    return best_x, fitness_values[best_index]
end


using Distributions

function mimic(pop_size, n_samples, n_elite, convergence_threshold, max_iterations, fitness_func)
    # initialize population
    population = rand(Bool, pop_size, n_samples)
    convergence = false
    iteration = 0
    elite_fitness = -Inf
    elite_individual = zeros(Bool, n_samples)
    
    while !convergence && iteration < max_iterations
        # evaluate fitness of each individual in the population
        fitness_values = [fitness_func(population[i,:]) for i in 1:pop_size]
        
        # check if the best individual has converged
        if maximum(fitness_values) > elite_fitness
            elite_fitness = maximum(fitness_values)
            elite_individual = population[argmax(fitness_values),:]
        end
        if elite_fitness >= convergence_threshold
            convergence = true
            break
        end
        
        # select elite individuals
        elite_indices = sortperm(fitness_values, rev=true)[1:n_elite]
        elite_population = population[elite_indices,:]
        
        # fit a multivariate distribution to the elite population
        elite_distribution = fit(MvNormal, transpose(elite_population))
        
        # sample new individuals from the distribution
        new_population = rand(elite_distribution, pop_size - n_elite)
        
        # combine elite and new individuals into new population
        population = vcat(elite_population, new_population)
        
        iteration += 1
    end
    
    return elite_individual, elite_fitness
end





using Distributions

function mimic2(pop_size, n_samples, n_elite, convergence_threshold, max_iterations, fitness_func)
    # initialize population
    population = rand(Bool, pop_size, n_samples)
    convergence = false
    iteration = 0
    elite_fitness = -Inf
    elite_individual = zeros(Bool, n_samples)
    
    while !convergence && iteration < max_iterations
        # evaluate fitness of each individual in the population
        fitness_values = [fitness_func(decode_individual(population[i,:])) for i in 1:pop_size]
        
        # check if the best individual has converged
        if maximum(fitness_values) > elite_fitness
            elite_fitness = maximum(fitness_values)
            elite_individual = population[argmax(fitness_values),:]
        end
        if elite_fitness >= convergence_threshold
            convergence = true
            break
        end
        
        # select elite individuals
        elite_indices = sortperm(fitness_values, rev=true)[1:n_elite]
        elite_population = population[elite_indices,:]
        
        # fit a multivariate distribution to the elite population
        elite_distribution = fit(MvNormal, transpose(elite_population))
        
        # sample new individuals from the distribution
        new_population = rand(elite_distribution, pop_size - n_elite)
        
        # combine elite and new individuals into new population
        population = vcat(elite_population, new_population)
        
        iteration += 1
    end
    
    # decode best individual and return best solution
    best_individual = decode_individual(elite_individual)
    return best_individual, elite_fitness
end

# helper function to decode an individual from binary to real-valued representation
function decode_individual(individual)
    return [sum(individual[(i-1)*10+1:i*10] .* 2.0 .^ (9:-1:0)) / 1023.0 for i in 1:10]
end


decode_individual (generic function with 1 method)

In [32]:
# Define the objective function
function objective(x)
    return (x[1]-2)^2 + (x[2]+1)^2 + (x[3]-4)^2
end


function energy_func(x)
    return x[1]^2 + x[2]^2
end

# Define the neighbor function
function neighbor_func(x)
    return x + randn(length(x))
end

# Set the initial state
initial_state = [1.0, 2.0]


# Set the initial temperature, cooling rate, and number of iterations
temperature = 10.0
cooling_rate = 0.95
num_iterations = 100

current_state, current_energy= simulated_annealing(initial_state, temperature, cooling_rate, num_iterations, energy_func, neighbor_func)



([-0.0715784466560857, -0.08720619739181895], 0.012728394889238995)

In [30]:

function energy_func(x)
    return x[1]^2 + x[2]^2
end

# Define the neighbor function
function neighbor_func(x)
    return x + randn(length(x))
end

# Set the initial state
initial_state = [1.0, 2.0]

max_iterations = 1000




current_state, current_energy= randomized_hill_climbing(initial_state, max_iterations, neighbor_func, energy_func)

([-0.03289688021838501, -0.019337930535940587], 0.0014561602855156343)

In [35]:


# Set the initial state
initial_state = [1.0, 2.0]

max_iterations = 1000

# Set the population size, number of genes, mutation rate, and number of generations
pop_size = 50
n_genes = 2
mutation_rate = 0.01
num_generations = 100
function energy_func(x)
    return x[1]^2 + x[2]^2
end


fitness_func = energy_func


# Call genetic_algorithm with the specified arguments
best_x, fv= genetic_algorithm(pop_size, n_genes, fitness_func, mutation_rate, num_generations)

best_x

2-element Vector{Int64}:
 1
 1

In [39]:



# Set the initial state
initial_state = [1.0, 2.0]
max_iterations = 1000


pop_size = 50
n_samples = 100
n_elite = 10
convergence_threshold = 0.01
max_iterations = 1000


function energy_func(x)
    return x[1]^2 + x[2]^2
end


fitness_func = energy_func



mimic2(pop_size, n_samples, n_elite, convergence_threshold, max_iterations, fitness_func)


([0.8592375366568915, 1.0, 0.4125122189638319, 0.30303030303030304, 0.592375366568915, 0.1544477028347996, 0.4789833822091887, 0.989247311827957, 0.31867057673509286, 0.2991202346041056], 1.7382891444002029)

In [55]:

using Flux
using Flux

# Define the neural network architecture
model = Chain(
    Dense(5, 10, relu),
    Dense(10, 1)
)

# Define the X and Y data
X = rand(100, 5)
Y = rand(100, 1)

# Define the loss function
loss(x, y) = Flux.mse(model(x), y)

# Define the initial weights for the network
weights = Flux.params(model)

# Define the annealing schedule
function schedule(t)
    T₀ = 10.0
    α = 0.99
    T₀ * α^t
end



schedule (generic function with 1 method)

In [57]:
using Flux, Random
using Flux

# Define the neural network architecture
model = Chain(
    Dense(5, 10, relu),
    Dense(10, 1)
)

# Define the X and Y data
X = rand(100, 5)
Y = rand(100, 1)

# Define the loss function
loss(x, y) = Flux.mse(model(x), y)


# Define the annealing schedule
function schedule(t)
    T₀ = 10.0
    α = 0.99
    T₀ * α^t
end


initial_weights = Flux.params(model)

function sa_optimize(loss, weightsi, schedule, max_iter)
    n_params = length(weightsi)
    current_loss = loss(X, Y; weights=weightsi)
    best_loss = current_loss
    best_weights = copy(weightsi)

    for i in 1:max_iter
        # Create a new candidate set of weights
        candidate_weights = [randn(size(w)) .* schedule(i) .+ w for w in weightsi]

        # Evaluate the loss for the candidate set of weights
        candidate_loss = loss(X, Y; weights=candidate_weights)

        # Determine whether to accept the candidate set of weights
        ΔE = candidate_loss - current_loss
        if rand() < exp(-ΔE / schedule(i))
            weightsi = candidate_weights
            current_loss = candidate_loss

            # Update the best set of weights and best loss
            if current_loss < best_loss
                best_loss = current_loss
                best_weights = copy(weightsi)
            end
        end
    end

    return best_weights
end



sa_optimize (generic function with 1 method)

In [58]:

# Fit the neural network using simulated annealing
best_weights = sa_optimize(loss, initial_weights, schedule, 1000)
Flux.loadparams!(model, best_weights)

# Evaluate the final loss
final_loss = loss(X, Y)
println("Final loss: $final_loss")


LoadError: MethodError: no method matching loss(::Matrix{Float64}, ::Matrix{Float64}; weights=Params([Float32[-0.37587023 0.39639837 … 0.14249869 0.03521703; 0.19096217 -0.5974775 … 0.31593025 0.041191675; … ; 0.545585 -0.39725122 … 0.4392625 0.17411132; -0.3754245 0.47545448 … 0.18314828 -0.017984018], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.7122445 -0.66971266 … -0.11024316 0.46141568], Float32[0.0]]))
[0mClosest candidates are:
[0m  loss(::Any, ::Any) at In[57]:15[91m got unsupported keyword argument "weights"[39m